Coding Exercise

- Data source: free API access from Coinapi.io
- API documentation: https://docs.coinapi.io/

### Set up

In [225]:
import requests
import numpy as np
import pandas as pd
import datetime as dt
import pytz
import time
import gdax

pd.options.mode.chained_assignment = None

In [196]:
'''
# limit range from 100 to 10000
limit = 1000

# Create trades table storing all trades
trades = pd.DataFrame()

# free api key (limit: 100 request/day) can be requested at: https://www.coinapi.io/pricing?apikey
api_key = '8E41560E-D4D2-49E2-9BDE-AD7ABC04005C' 

url = 'https://rest.coinapi.io/v1/trades/COINBASE_SPOT_BTC_USD/latest?limit=' + str(limit)
headers = {'X-CoinAPI-Key' : api_key}
'''

"\n# limit range from 100 to 10000\nlimit = 1000\n\n# Create trades table storing all trades\ntrades = pd.DataFrame()\n\n# free api key (limit: 100 request/day) can be requested at: https://www.coinapi.io/pricing?apikey\napi_key = '8E41560E-D4D2-49E2-9BDE-AD7ABC04005C' \n\nurl = 'https://rest.coinapi.io/v1/trades/COINBASE_SPOT_BTC_USD/latest?limit=' + str(limit)\nheaders = {'X-CoinAPI-Key' : api_key}\n"

In [197]:
'''
url = 'http://marketdata.wavesplatform.com/api/trades/BTC/USD/500'
response = requests.get(url)
trades_new = pd.DataFrame(response.json())
trades_new['datetime'] = trades_new['timestamp'].apply(lambda t: dt.datetime.utcfromtimestamp(t/1000))
trades_new.head()
'''

"\nurl = 'http://marketdata.wavesplatform.com/api/trades/BTC/USD/500'\nresponse = requests.get(url)\ntrades_new = pd.DataFrame(response.json())\ntrades_new['datetime'] = trades_new['timestamp'].apply(lambda t: dt.datetime.utcfromtimestamp(t/1000))\ntrades_new.head()\n"

In [198]:
# Stole code from https://github.com/danpaquin/gdax-python

#
# GDAX/PublicClient.py
# Daniel Paquin
#
# For public requests to the GDAX exchange

import requests


class PublicClient(object):
    """GDAX public client API.
    All requests default to the `product_id` specified at object
    creation if not otherwise specified.
    Attributes:
        url (Optional[str]): API URL. Defaults to GDAX API.
    """

    def __init__(self, api_url='https://api.gdax.com', timeout=30):
        """Create GDAX API public client.
        Args:
            api_url (Optional[str]): API URL. Defaults to GDAX API.
        """
        self.url = api_url.rstrip('/')
        self.timeout = timeout

    def _get(self, path, params=None):
        """Perform get request"""

        r = requests.get(self.url + path, params=params, timeout=self.timeout)
        # r.raise_for_status()
        return r.json()

    def get_products(self):
        """Get a list of available currency pairs for trading.
        Returns:
            list: Info about all currency pairs. Example::
                [
                    {
                        "id": "BTC-USD",
                        "display_name": "BTC/USD",
                        "base_currency": "BTC",
                        "quote_currency": "USD",
                        "base_min_size": "0.01",
                        "base_max_size": "10000.00",
                        "quote_increment": "0.01"
                    }
                ]
        """
        return self._get('/products')

    def get_product_order_book(self, product_id, level=1):
        """Get a list of open orders for a product.
        The amount of detail shown can be customized with the `level`
        parameter:
        * 1: Only the best bid and ask
        * 2: Top 50 bids and asks (aggregated)
        * 3: Full order book (non aggregated)
        Level 1 and Level 2 are recommended for polling. For the most
        up-to-date data, consider using the websocket stream.
        **Caution**: Level 3 is only recommended for users wishing to
        maintain a full real-time order book using the websocket
        stream. Abuse of Level 3 via polling will cause your access to
        be limited or blocked.
        Args:
            product_id (str): Product
            level (Optional[int]): Order book level (1, 2, or 3).
                Default is 1.
        Returns:
            dict: Order book. Example for level 1::
                {
                    "sequence": "3",
                    "bids": [
                        [ price, size, num-orders ],
                    ],
                    "asks": [
                        [ price, size, num-orders ],
                    ]
                }
        """

        # Supported levels are 1, 2 or 3
        level = level if level in range(1, 4) else 1
        return self._get('/products/{}/book'.format(str(product_id)), params={'level': level})

    def get_product_ticker(self, product_id):
        """Snapshot about the last trade (tick), best bid/ask and 24h volume.
        **Caution**: Polling is discouraged in favor of connecting via
        the websocket stream and listening for match messages.
        Args:
            product_id (str): Product
        Returns:
            dict: Ticker info. Example::
                {
                  "trade_id": 4729088,
                  "price": "333.99",
                  "size": "0.193",
                  "bid": "333.98",
                  "ask": "333.99",
                  "volume": "5957.11914015",
                  "time": "2015-11-14T20:46:03.511254Z"
                }
        """
        return self._get('/products/{}/ticker'.format(str(product_id)))

    def get_product_trades(self, product_id, before='', after='', limit=100, result=[]):
        """List the latest trades for a product.
        Args:
             product_id (str): Product
             before (Optional[str]): start time in ISO 8601
             after (Optional[str]): end time in ISO 8601
             limit (Optional[int]): the desired number of trades (can be more than 100,
                          automatically paginated)
             results (Optional[list]): list of results that is used for the pagination
        Returns:
             list: Latest trades. Example::
                 [{
                     "time": "2014-11-07T22:19:28.578544Z",
                     "trade_id": 74,
                     "price": "10.00000000",
                     "size": "0.01000000",
                     "side": "buy"
                 }, {
                     "time": "2014-11-07T01:08:43.642366Z",
                     "trade_id": 73,
                     "price": "100.00000000",
                     "size": "0.01000000",
                     "side": "sell"
         }]
        """
        url = self.url + '/products/{}/trades'.format(str(product_id))
        params = {}

        if before:
            params['before'] = str(before)
        if after:
            params['after'] = str(after)
        if limit and limit < 100:
            # the default limit is 100
            # we only add it if the limit is less than 100
            params['limit'] = limit

        r = requests.get(url, params=params)
        # r.raise_for_status()

        result.extend(r.json())

        if 'cb-after' in r.headers and limit is not len(result):
            # update limit
            limit = limit - len(result)
            if limit <= 0:
                return result

            # TODO: need a way to ensure that we don't get rate-limited/blocked
            # time.sleep(0.4)
            return self.get_product_trades(product_id=product_id, after=r.headers['cb-after'], limit=limit, result=result)

        return result

    def get_product_historic_rates(self, product_id, start=None, end=None,
                                   granularity=None):
        """Historic rates for a product.
        Rates are returned in grouped buckets based on requested
        `granularity`. If start, end, and granularity aren't provided,
        the exchange will assume some (currently unknown) default values.
        Historical rate data may be incomplete. No data is published for
        intervals where there are no ticks.
        **Caution**: Historical rates should not be polled frequently.
        If you need real-time information, use the trade and book
        endpoints along with the websocket feed.
        The maximum number of data points for a single request is 200
        candles. If your selection of start/end time and granularity
        will result in more than 200 data points, your request will be
        rejected. If you wish to retrieve fine granularity data over a
        larger time range, you will need to make multiple requests with
        new start/end ranges.
        Args:
            product_id (str): Product
            start (Optional[str]): Start time in ISO 8601
            end (Optional[str]): End time in ISO 8601
            granularity (Optional[str]): Desired time slice in seconds
        Returns:
            list: Historic candle data. Example::
                [
                    [ time, low, high, open, close, volume ],
                    [ 1415398768, 0.32, 4.2, 0.35, 4.2, 12.3 ],
                    ...
                ]
        """
        params = {}
        if start is not None:
            params['start'] = start
        if end is not None:
            params['end'] = end
        if granularity is not None:
            acceptedGrans = [60, 300, 900, 3600, 21600, 86400]
            if granularity not in acceptedGrans:
                newGranularity = min(acceptedGrans, key=lambda x:abs(x-granularity))
                print(granularity,' is not a valid granularity level, using',newGranularity,' instead.')
                granularity = newGranularity
            params['granularity'] = granularity

        return self._get('/products/{}/candles'.format(str(product_id)), params=params)

    def get_product_24hr_stats(self, product_id):
        """Get 24 hr stats for the product.
        Args:
            product_id (str): Product
        Returns:
            dict: 24 hour stats. Volume is in base currency units.
                Open, high, low are in quote currency units. Example::
                    {
                        "open": "34.19000000",
                        "high": "95.70000000",
                        "low": "7.06000000",
                        "volume": "2.41000000"
                    }
        """
        return self._get('/products/{}/stats'.format(str(product_id)))

    def get_currencies(self):
        """List known currencies.
        Returns:
            list: List of currencies. Example::
                [{
                    "id": "BTC",
                    "name": "Bitcoin",
                    "min_size": "0.00000001"
                }, {
                    "id": "USD",
                    "name": "United States Dollar",
                    "min_size": "0.01000000"
                }]
        """
        return self._get('/currencies')

    def get_time(self):
        """Get the API server time.
        Returns:
            dict: Server time in ISO and epoch format (decimal seconds
                since Unix epoch). Example::
                    {
                        "iso": "2015-01-07T23:47:25.201Z",
                        "epoch": 1420674445.201
                    }
        """
        return self._get('/time')

In [ ]:
def __init__(self, api_url='https://api.gdax.com', timeout=30):
        """Create GDAX API public client.
        Args:
            api_url (Optional[str]): API URL. Defaults to GDAX API.
        """
        self.url = 'https://api.gdax.com'.rstrip('/')
        self.timeout = timeout


In [441]:
'https://api.gdax.com'.rstrip('/')

'https://api.gdax.com'

In [538]:
def get_product_trades(product_id, before='', after='', limit=100, result=[]):
    """List the latest trades for a product.
    Args:
         product_id (str): Product
         before (Optional[str]): start time in ISO 8601
         after (Optional[str]): end time in ISO 8601
         limit (Optional[int]): the desired number of trades (can be more than 100,
                      automatically paginated)
         results (Optional[list]): list of results that is used for the pagination
    Returns:
         list: Latest trades. Example::
             [{
                 "time": "2014-11-07T22:19:28.578544Z",
                 "trade_id": 74,
                 "price": "10.00000000",
                 "size": "0.01000000",
                 "side": "buy"
             }, {
                 "time": "2014-11-07T01:08:43.642366Z",
                 "trade_id": 73,
                 "price": "100.00000000",
                 "size": "0.01000000",
                 "side": "sell"
     }]
    """
    url = 'https://api.gdax.com/products/{}/trades'.format(str(product_id))
    params = {}

    if before:
        params['before'] = str(before)
    if after:
        params['after'] = str(after)
    if limit and limit < 100:
        # the default limit is 100
        # we only add it if the limit is less than 100
        params['limit'] = limit

    r = requests.get(url, params=params)
    
    result.extend(r.json())

    if 'cb-after' in r.headers and limit is not len(result):
        # update limit
        limit = limit - len(result)
        if limit <= 0:
            return result

        # TODO: need a way to ensure that we don't get rate-limited/blocked
        time.sleep(0.4)
        return get_product_trades(product_id=product_id, after=r.headers['cb-after'], limit=limit, result=result)

    return result

In [564]:
trades_raw = get_product_trades(product_id='BTC-USD', limit=100)

In [565]:
trades_new = pd.DataFrame(trades_raw)

In [566]:
trades_new

,price,side,size,time,trade_id
0,7291.87000000,sell,0.03361339,2018-07-19T02:10:56.812Z,46737960
1,7291.87000000,sell,6.19671337,2018-07-19T02:10:51.42Z,46737959
2,7291.87000000,sell,0.01694383,2018-07-19T02:10:51.259Z,46737958
3,7291.87000000,sell,0.53781426,2018-07-19T02:10:49.129Z,46737957
4,7291.86000000,buy,0.17402623,2018-07-19T02:10:44.921Z,46737956
5,7291.87000000,sell,0.03542963,2018-07-19T02:10:44.305Z,46737955
6,7291.86000000,buy,0.05776262,2018-07-19T02:10:44.097Z,46737954
7,7291.87000000,sell,0.00122948,2018-07-19T02:10:43.901Z,46737953
8,7291.87000000,sell,0.00846168,2018-07-19T02:10:41.416Z,46737952
9,7291.86000000,buy,0.00330000,2018-07-19T02:10:32.9Z,46737951


In [567]:
len(trades_new['trade_id'].unique())

218

In [625]:
r = requests.get('https://api.gdax.com/products/BTC-USD/trades')
a = pd.DataFrame(r.json()).set_index('trade_id')
time.sleep(10)
r = requests.get('https://api.gdax.com/products/BTC-USD/trades')
b = pd.DataFrame(r.json()).set_index('trade_id')

In [632]:
merged = a.reset_index().merge(b.reset_index(), indicator=True, how='outer').set_index('trade_id')
merged

,price,side,size,time,_merge
trade_id,,,,,
46740987,7327.31000000,sell,0.00570384,2018-07-19T03:04:35.121Z,both
46740986,7327.31000000,sell,0.00448265,2018-07-19T03:04:34.905Z,both
46740985,7327.30000000,buy,0.10000000,2018-07-19T03:04:31.675Z,both
46740984,7327.31000000,sell,0.01631996,2018-07-19T03:04:23.535Z,both
46740983,7327.30000000,buy,0.03770655,2018-07-19T03:04:21.716Z,both
46740982,7328.46000000,buy,0.01250000,2018-07-19T03:04:21.716Z,both
46740981,7328.56000000,buy,0.02731009,2018-07-19T03:04:21.716Z,both
46740980,7328.56000000,buy,0.00098336,2018-07-19T03:04:21.716Z,both
46740979,7328.56000000,buy,0.00032664,2018-07-19T03:04:17.533Z,both


In [685]:
merged[merged['_merge'] == 'right_only'].drop('_merge', axis=1)

,price,side,size,time
trade_id,,,,
46740993,7327.31000000,sell,0.03283565,2018-07-19T03:04:46.575Z
46740992,7327.30000000,buy,0.02715665,2018-07-19T03:04:46.101Z
46740991,7327.31000000,sell,0.00353207,2018-07-19T03:04:44.796Z
46740990,7327.31000000,sell,0.02675332,2018-07-19T03:04:43.997Z
46740989,7327.30000000,buy,0.00140000,2018-07-19T03:04:41.361Z
46740988,7327.31000000,sell,0.05500000,2018-07-19T03:04:38.736Z


In [696]:
def parties_assign(n_parties, table1, table2=None):
    """
    Assign buyers and sellers to each trade

    args:
        n_parties: total number of counter parties
        table: the table of trades
    """
    # print('parties_assign received table:')

    parties_list = []
    
    if table2 is not None:
        merged = table1.reset_index().merge(table2.reset_index(),
                                            indicator=True,
                                            how='outer').set_index('trade_id')
        new_rows = merged[merged['_merge'] == 'right_only'].drop('_merge',
                                                                 axis=1)
        
        if len(merged) > 0:
            
            for i in np.arange(len(new_rows)):
                parties_list.append(np.random.choice(10, 2, replace=False))
                
            parties_list = np.array(parties_list)
            
            new_rows['Buyer'] = parties_list[:,0]
            new_rows['Seller'] = parties_list[:,1]

            table1 = pd.concat([table1, new_rows])

    else:
        for i in np.arange(len(table1)):
            parties_list.append(np.random.choice(n_parties, 2, replace=False))
        
        parties_list = np.array(parties_list)

        table1['Buyer'] = parties_list[:, 0]
        table1['Seller'] = parties_list[:, 1]
                
    return table1

In [697]:
parties_assign(10, a, b)

,price,side,size,time,Buyer,Seller
trade_id,,,,,,
46740987,7327.31000000,sell,0.00570384,2018-07-19T03:04:35.121Z,0,4
46740986,7327.31000000,sell,0.00448265,2018-07-19T03:04:34.905Z,9,8
46740985,7327.30000000,buy,0.10000000,2018-07-19T03:04:31.675Z,1,0
46740984,7327.31000000,sell,0.01631996,2018-07-19T03:04:23.535Z,3,7
46740983,7327.30000000,buy,0.03770655,2018-07-19T03:04:21.716Z,9,6
46740982,7328.46000000,buy,0.01250000,2018-07-19T03:04:21.716Z,3,5
46740981,7328.56000000,buy,0.02731009,2018-07-19T03:04:21.716Z,8,3
46740980,7328.56000000,buy,0.00098336,2018-07-19T03:04:21.716Z,0,6
46740979,7328.56000000,buy,0.00032664,2018-07-19T03:04:17.533Z,5,6


In [598]:
pd.concat?

In [599]:
e = pd.concat([a,d]).drop_duplicates()

In [600]:
e

,price,side,size,time
trade_id,,,,
46738094,7299.99000000,buy,0.00173701,2018-07-19T02:12:31.628Z
46738093,7300.00000000,sell,0.00518094,2018-07-19T02:12:29.217Z
46738092,7299.99000000,buy,0.00260000,2018-07-19T02:12:28.61Z
46738091,7300.00000000,sell,0.00252300,2018-07-19T02:12:28.298Z
46738090,7300.00000000,sell,0.00320453,2018-07-19T02:12:27.32Z
46738089,7300.00000000,sell,0.13437727,2018-07-19T02:12:23.688Z
46738088,7300.00000000,sell,0.02003817,2018-07-19T02:12:21.825Z
46738087,7300.00000000,sell,0.00320453,2018-07-19T02:12:19.51Z
46738086,7299.99000000,buy,0.11707468,2018-07-19T02:12:18.244Z


In [594]:
len(c.index.unique())

200

In [573]:
len(pd.DataFrame(r.json())['trade_id'].unique())

100

There is some delay in the live data, so we will use a delayed window.

In [318]:
# Delayed minutes defined by user
delay = 3

# Get current UTC time
now = dt.datetime.utcnow()

# Get current minute
current_minute = dt.datetime(now.year,now.month,now.day,now.hour,now.minute)

# Get desired time window
end_time = current_minute - dt.timedelta(minutes=delay)
start_time = current_minute - dt.timedelta(minutes=delay+1)

In [345]:
def iso_converter(s):
    if len(s) == 20:
        t = dt.datetime.strptime(s, "%Y-%m-%dT%H:%M:%SZ")
    else:
        t = dt.datetime.strptime(s, "%Y-%m-%dT%H:%M:%S.%fZ")

    return t

# Convert string to datetime object
trade_time = trades_new['time'].apply(iso_converter)

# Select rows between 1 min ago and 2 min ago
mask = (trade_time < end_time) & (trade_time > start_time)
trades_selected = trades_new[mask]

In [204]:
trades_selected.head()

,price,side,size,time,trade_id
67,497.05000000,sell,1.86177929,2018-07-18T09:36:38.295Z,37495997
68,497.05000000,sell,0.02112071,2018-07-18T09:36:38.295Z,37495996
69,496.62000000,buy,5.30054890,2018-07-18T09:36:35.181Z,37495995
70,496.78000000,buy,0.67790000,2018-07-18T09:36:35.181Z,37495994
71,496.78000000,buy,0.04000000,2018-07-18T09:36:35.181Z,37495993


In [223]:
def counterparty_assign(n_parties, table):
    '''
    args:
        n: total number of counterparties
        table: the table of trades
    '''
    parties_list = []

    for i in np.arange(len(table)):
        parties_list.append(np.random.choice(n_parties, 2, replace=False))

    parties_list = np.array(parties_list)

    table['Buyer'] = parties_list[:,0]
    table['Seller'] = parties_list[:,1]
    
    return table


In [226]:
trades_selected = counterparty_assign(10, trades_selected)

,price,side,size,time,trade_id,Buyer,Seller
67,497.05000000,sell,1.86177929,2018-07-18T09:36:38.295Z,37495997,5,0
68,497.05000000,sell,0.02112071,2018-07-18T09:36:38.295Z,37495996,0,6
69,496.62000000,buy,5.30054890,2018-07-18T09:36:35.181Z,37495995,4,3
70,496.78000000,buy,0.67790000,2018-07-18T09:36:35.181Z,37495994,3,1
71,496.78000000,buy,0.04000000,2018-07-18T09:36:35.181Z,37495993,0,2
72,496.91000000,buy,2.89641622,2018-07-18T09:36:35.181Z,37495992,0,5
73,496.92000000,buy,0.10860400,2018-07-18T09:36:35.181Z,37495991,7,6
74,496.92000000,buy,0.01000000,2018-07-18T09:36:35.181Z,37495990,8,7
75,497.04000000,buy,0.01048000,2018-07-18T09:36:35.181Z,37495989,1,0
76,497.04000000,buy,1.71000000,2018-07-18T09:36:35.181Z,37495988,0,4


In [256]:
balances = pd.DataFrame({'Party': range(10), 'Balance': 100000}).set_index('Party')

In [407]:
def retrieve_data(prod_id='BTC-USD', lim=1000):
    """
    args:
        prod_id: product ID, eg. BTC-USD, ETH-USD
        lim: API rate limit

    return: new data retrieved from GDAX
    """

    public_client = PublicClient()

    print('Data collection started...')

    # Get raw data
    raw_data = public_client.get_product_trades(product_id=prod_id,
                                                limit=lim)
    print(str(len(raw_data)) + ' data points have been collected...')

    while len(raw_data) < lim:
        # Get raw data
        raw_data = public_client.get_product_trades(product_id=prod_id,
                                                    limit=lim)
        print(str(len(raw_data)) + ' data points have been collected...')
        time.sleep(0.4) # GDAX rate limit: 3 requests per second

    print('Data collection complete!')
    print('Last retrieval time (UTC): ' + str(dt.datetime.utcnow()))

    # Convert to dataframe and keep desired amount of data
    df = pd.DataFrame(raw_data).set_index('trade_id')[0:lim]

    # Convert string to datetime object
    df['time'] = df['time'].apply(iso_converter)

    return df

In [412]:
trades_BTC = retrieve_data('BTC-USD', 3000)

Data collection started...
4800 data points have been collected...
Data collection complete!
Last retrieval time (UTC): 2018-07-19 01:36:53.074917


In [413]:
trades_BTC

,price,side,size,time
trade_id,,,,
37496064,497.10000000,sell,0.86840000,2018-07-18 09:40:15.340
37496063,497.08000000,sell,0.01000000,2018-07-18 09:40:15.340
37496062,497.03000000,buy,1.03097447,2018-07-18 09:39:30.538
37496061,497.03000000,buy,0.01000000,2018-07-18 09:39:30.538
37496060,497.03000000,buy,0.10860600,2018-07-18 09:39:30.538
37496059,497.05000000,buy,1.07000000,2018-07-18 09:39:30.538
37496058,497.05000000,buy,0.01088000,2018-07-18 09:39:30.538
37496057,497.05000000,buy,0.02000000,2018-07-18 09:39:30.538
37496056,497.05000000,buy,0.03010000,2018-07-18 09:39:29.832
